In [1]:
import os
import pickle
import numpy as np
import torch

from spenderq import load_model
from spenderq import util as U
from spenderq import lyalpha as LyA

In [16]:
from astropy.table import Table, vstack

In [2]:
# --- plotting --- 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False
%matplotlib inline

In [6]:
redshifts, tids = [], []
z_absorbers = []
for ibatch in range(85): 
    # load batch
    #with open('/tigress/chhahn/spender_qso/train/edr.lfsc.rsnr.sa1p1sb0p8.i%i_%i.pkl' % (niter, ibatch), "rb") as f:
    with open('/tigress/chhahn/spender_qso/train/DESI.edr.qso_highz_%i.pkl' % ibatch, 'rb') as f: 
        _, _, z, target_id, _, _ = pickle.load(f)
        
    # read absorbers
    i_absorb, z_absorb = [], [] 
    with open('/tigress/chhahn/spender_qso/absorbers/MgII.DESI.edr.qso_highz_%i.pkl' % ibatch, 'rb') as f: 
        out = pickle.load(f)
        i_absorb.append(np.concatenate([np.repeat(_o[0], len(_o[1])) for _o in out]))
        z_absorb.append([_o[1] for _o in out])
    i_absorb = np.concatenate(np.array(i_absorb))
    z_absorb = np.concatenate(z_absorb[0])
    z_absorbers.append(z_absorb)
    
    tids.append(np.array(target_id)[i_absorb])
    redshifts.append(np.array(z)[i_absorb])

tids = np.concatenate(tids)
redshifts = np.concatenate(redshifts)
z_absorbers = np.concatenate(z_absorbers)

In [9]:
mgii_absorbers = Table()
mgii_absorbers['ABSORBER'] = np.repeat('MgII', len(tids))
mgii_absorbers['TARGETID'] = tids
mgii_absorbers['Z_ABSORBER'] = z_absorbers
mgii_absorbers['Z_QSO'] = redshifts

In [12]:
print('%i MgII absorbers' % len(mgii_absorbers))

5709 MgII absorbers


In [19]:
redshifts, tids = [], []
z_absorbers = []
for ibatch in range(85): 
    # load batch
    #with open('/tigress/chhahn/spender_qso/train/edr.lfsc.rsnr.sa1p1sb0p8.i%i_%i.pkl' % (niter, ibatch), "rb") as f:
    with open('/tigress/chhahn/spender_qso/train/DESI.edr.qso_highz_%i.pkl' % ibatch, 'rb') as f: 
        _, _, z, target_id, _, _ = pickle.load(f)
    

    # read absorbers
    i_absorb, z_absorb = [], [] 
    with open('/tigress/chhahn/spender_qso/absorbers/CIV.DESI.edr.qso_highz_%i.pkl' % ibatch, 'rb') as f: 
        out = pickle.load(f)
        i_absorb.append(np.concatenate([np.repeat(_o[0], len(_o[1])) for _o in out]))
        z_absorb.append([_o[1] for _o in out])
    i_absorb = np.concatenate(np.array(i_absorb))
    z_absorb = np.concatenate(z_absorb[0])
    z_absorbers.append(z_absorb)

    tids.append(np.array(target_id)[i_absorb])
    redshifts.append(np.array(z)[i_absorb])

tids = np.concatenate(tids)
redshifts = np.concatenate(redshifts)
z_absorbers = np.concatenate(z_absorbers)

In [20]:
civ_absorbers = Table()
civ_absorbers['ABSORBER'] = np.repeat('CIV', len(tids))
civ_absorbers['TARGETID'] = tids
civ_absorbers['Z_ABSORBER'] = z_absorbers
civ_absorbers['Z_QSO'] = redshifts

In [21]:
print(len(civ_absorbers))

8018


In [22]:
all_absorbers = vstack([mgii_absorbers, civ_absorbers])

In [24]:
all_absorbers.write('/tigress/chhahn/spender_qso/absorbers/DESI.edr.qso_highz.absorbers.hdf5')

/home/chhahn/.conda/envs/gqp/lib/python3.11/site-packages/astropy/io/misc/hdf5.py:281: UserWarning: table path was not set via the path= argument; using default path __astropy_table__
  warnings.warn(
